# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
import json
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sitka,57.0531,-135.3300,284.09,76,100,4.63,US,1718138323
1,1,longyearbyen,78.2186,15.6401,277.06,81,75,4.63,SJ,1718138324
2,2,new norfolk,-42.7826,147.0587,279.22,84,66,2.66,AU,1718138084
3,3,okondja,-0.6549,13.6753,297.30,63,47,0.45,GA,1718138325
4,4,newman,37.3138,-121.0208,307.34,13,0,3.60,US,1718138051


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)
    

# Display the map
map_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df.loc[city_data_df["Cloudiness"]== 0,:]

# Drop any rows with null values
filtered_df = criteria_df.dropna()

# Display sample data
filtered_df.sample(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
414,414,pacific grove,36.6177,-121.9166,304.62,74,0,3.58,US,1718138559
377,377,mount isa,-20.7333,139.5000,280.02,70,0,0.00,AU,1718138539
190,190,shafter,35.5005,-119.2718,313.90,15,0,3.69,US,1718138436
554,554,tasqala,51.1167,50.3167,294.84,56,0,3.48,KZ,1718138637
353,353,fort mcmurray,56.7268,-111.3810,292.92,42,0,1.54,CA,1718138321
89,89,khanty-mansiysk,61.0042,69.0019,290.21,59,0,1.00,RU,1718138379
300,300,remire-montjoly,4.9167,-52.2667,302.17,79,0,5.14,GF,1718138496
73,73,hawaiian paradise park,19.5933,-154.9731,301.50,65,0,4.63,US,1718138370
394,394,al jahra',29.3375,47.6581,311.27,12,0,5.67,KW,1718138548
312,312,sur,22.5667,59.5289,305.37,48,0,7.02,OM,1718138502


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sitka,US,57.0531,-135.3300,76,
1,longyearbyen,SJ,78.2186,15.6401,81,
2,new norfolk,AU,-42.7826,147.0587,84,
3,okondja,GA,-0.6549,13.6753,63,
4,newman,US,37.3138,-121.0208,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
#https://api.geoapify.com/v2/places?categories=accommodation.hotel&filter=circle:111.4917562,65.9516363,10000&bias=proximity:111.4917562,65.9516363&limit=20&apiKey=YOUR_API_KEY

# Set parameters to search for a hotel

radius = 10000
categories ="accommodation.hotel"
limit = 1

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude =row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    filter = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params = {
    "categories":categories,
    "filter":filter,
    "bias": bias,
    "limit": limit,
    "apiKey":geoapify_key
} 

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
            hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            
        # Log the search results
            print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
okondja - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
luau - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
olonkinbyen - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
arteche - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
umm kaddadah - nearest hotel: No hotel found
isola di capo rizzuto - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
lukolela - nearest hotel: No hotel found
zhanaozen - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
awgu - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
enewetak - nearest hotel: N

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sitka,US,57.0531,-135.3300,76,Aspen Suites Hotel
1,longyearbyen,SJ,78.2186,15.6401,81,The Vault
2,new norfolk,AU,-42.7826,147.0587,84,Woodbridge on the Derwent
3,okondja,GA,-0.6549,13.6753,63,No hotel found
4,newman,US,37.3138,-121.0208,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Marker Size",
    scale = 0.8,
    hover_cols=["Hotel Name", "City", "Country"],
    color="City"
)

# Display the map
map_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)